# URLs

In [ ]:
from django.contrib import admin
from django.urls import path
from boards import views

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', views.home, name='home'),
    path('boards/<int:pk>/', views.board_topics, name='board_topics'),
]

URL dispatcher và URLconf là những parts cần thiết cho Django app

Note: Một project có `urls.py` đặc biệt gọi là __root URLconf__. (root gốc)

Bạn có thể tạo thêm nhiều file urls.py ở các app nhưng root vẫn là `urls.py` nằm trong folder project chính

In [ ]:
def path(regex, view, kwargs=None, name=None)

__regex__: Một expression thông thường cho các đường dẫn URL dưới dạng string. Nếu ta muốn chèn thêm tham số vào urls thì hãy dùng < int:pk > hoặc < slug:slug >

__view__: function view được dùng để xử lý request của user cho một URL được matched. File view này có thể trả về function __django.conf.urls.include__, tham chiếu sang file __urls.py__

__kwargs__: Một tham số tùy ý được passed vào file view. Thông thường ko sử dụng

__name__: tên gọi cho url, một tính năng quan trọng. Luôn luôn gọi tên cho đường dẫn URLs. Với nó, bạn có thể thay đổi URL đặc trưng trong cả project của bạn chỉ bằng cách change Regex

# Test path 404

Bạn muốn test một đường dẫn xảy lỗi 404

- Nhớ sử dụng `ObjectDoesNotExist` trong thư viện `django.core.exceptions`

- Nhớ sử dụng raise `Http404` trong package `django.http`

Khi ta đang tạo trang web trong điều kiện `DEBUG=False`, visitor sẽ thấy __500 Internal Server Error__

Nên hãy trả về __404 error__ trong các functions view

Trong file views.py, các functions view dùng trả về response cho một đường dẫn. Ta hãy sử dụng

In [ ]:
    try:
        board = Board.objects.get(pk=pk)
    except ObjectDoesNotExist:
        raise Http404

Trong file test.py, hãy tạo một bài test cho đường dẫn không tồn tại

In [ ]:
    def test_board_topics_view_not_found_status_code(self):
        url = reverse('board_topics', kwargs={'pk': 100})
        response = self.client.get(url)
        self.assertEquals(response.status_code, 404)


# List of Useful URL Patterns

Trick part is the regex

__Primary Key AutoField__

Regex: `<int:pk>`

Example: `path("questions/<int:pk>", views.question, name='question')`

__Slug Field__

Regex: `<slug:slug>`

Example: `path("posts/<slug:slug>/", views.post, name='post')`

__Slug Field with Primary Key__

Regex: `<slug:slug>-<int:pk>`

Example: `path("posts/<slug:slug>-<int:pk>/")`

__Django User Username__

Regex: `username`

Example: `path("<username>/blog/")`

__Year__

Regex: `<int:year>`

Example: `path('articles/<int:year>/', views.year_archive),`

`Year / Month`

Regex: `<int:year>/<int:month>)
`

Example: `path('articles/<int:year>/<int:month>/', views.month_archive)`

# Reusable templates

Refactor HTML templates, creating a __master page__ ( tạo một trang nền ). Và sử dụng reusable templates

- Ko cần copy-paste template vào các trang khác
- Tiết kiệm thời gian

Tạo trang nền => Đặt tên là `base.html` trong templates folder

Mỗi template ta muốn kế thừa `master page`. Ta sẽ sử dụng __extend__ trang base.html

In [ ]:
{% load static %}<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title>{% block title %}Django Boards{% endblock %}</title>
    <link rel="stylesheet" href="{% static 'css/bootstrap.min.css' %}">
  </head>
  <body>
    <div class="container">
      <ol class="breadcrumb my-4">
        {% block breadcrumb %}
        {% endblock %}
      </ol>
      {% block content %}
      {% endblock %}
    </div>
  </body>
</html>

- Các trang con kế thừa trang nền

- {% block %} tag là để cho chúng ta sử dụng space tùy chỉnh cho các trang con

- {% block title %} Django Boards {% endblock %}

Ở đây ta đang đặt default cho các trang con cũng mang tên Django boards. Ta có thể set value lại theo ý mình thích



__{% extends 'base.html' %}__

    Nói với trang template rằng nó sẽ kế thứa trang 'base.html'

In [ ]:
# topics.html

{% extends 'base.html' %}

{% block title %}
  {{ board.name }} - {{ block.super }}
{% endblock %}

{% block breadcrumb %}
  <li class="breadcrumb-item"><a href="{% url 'home' %}">Boards</a></li>
  <li class="breadcrumb-item active">{{ board.name }}</li>
{% endblock %}

{% block content %}
    <!-- just leaving it empty for now. we will add core here soon. -->
{% endblock %}

Ta hãy để ý `{% block title %}` sẽ không lấy default là Django boards

Nó sẽ xét thêm {{ board.name }} tức name của object `board`. Ngoài ra nếu ta muốn sử dụng lại giá trị default (`Django boards`), ta có thể sử dụng tag `{{ block.super }}`

# Font Google

Ta có thể đổi fonts đa dạng cho các chữ bằng cách

Vao địa chỉ https://fonts.google.com/

Kiếm font bạn yêu thích

# Forms

Forms được sử dụng để giải quyết các input cho các user

Quá trình xử lý forms là phức tạp vì nó liên quan nhiều layers của một app

Các vấn đề cần quan tâm

- proper data type (integer, float, date,...)

=> Validate data bất kể business logic của app ntn

- clean, sanitize the data để tránh các trường hợp SQL Injection và XSS attacks



Tin tốt là Django Forms API làm quá trình dễ xử lý hơn. `automating a good chunk of this work`

Kết quả cuối sẽ được đảm bảo secure (an toàn hơn)

# How to implement a Form

__ REMEMBER: underlying details of form processing. __

In [ ]:
# templates/new_topic.html
{% extends 'base.html' %}

{% block title %}Start a New Topic{% endblock %}

{% block breadcrumb %}
  <li class="breadcrumb-item"><a href="{% url 'home' %}">Boards</a></li>
  <li class="breadcrumb-item"><a href="{% url 'board_topics' board.pk %}">{{ board.name }}</a></li>
  <li class="breadcrumb-item active">New topic</li>
{% endblock %}

{% block content %}
  <form method="post">
    {% csrf_token %}
    <div class="form-group">
      <label for="id_subject">Subject</label>
      <input type="text" class="form-control" id="id_subject" name="subject">
    </div>
    <div class="form-group">
      <label for="id_message">Message</label>
      <textarea class="form-control" id="id_message" name="message" rows="5"></textarea>
    </div>
    <button type="submit" class="btn btn-success">Post</button>
  </form>
{% endblock %}

Request có hai phương thức:

- GET: thường sử dụng cho các request không gửi dữ liệu, không truy xuất dũ liệu server...

- POST: được sử dụng thường xuyên khi ta gửi dữ liệu lên server

__csrf_token__: token bảo mật của Django. Django sẽ bảo vệ tất cả request 'POST' 

Để truy xuất hai data từ các request.post bên code python

Ví dụ: 
- subject = request.POST['subject']
- message = request.POST['message']

NOTE: trong code html, khi ta render các objects liên kết với nhau. Ví dụ topic là foreign key của board. Ta có thể truy xuất các objects `topic` của một board như sau trong code html:

`{{ board.topics.all }}`

Lý do ta sử dụng `board.topics.all` thay vì `board.topics` là vì `board.topics.all` là một Related Manager, giống như là __Model Manager__ => giống như `board.objects` property

Nên, thay vì trả tất cả topics liên kết với board, ta phải trả về `board.topics.all()`

Nếu mún filter thêm ta hãy dùng `board.topics.filter(subject__contains='...')`

In [ ]:
class NewTopicTests(TestCase):
    def setUp(self):
        Board.objects.create(name='Django', description='Django board.')
        User.objects.create_user(username='john', email='john@doe.com', password='123')  # <- included this line here

    # ...

    def test_csrf(self):
        url = reverse('new_topic', kwargs={'pk': 1})
        response = self.client.get(url)
        self.assertContains(response, 'csrfmiddlewaretoken')

    def test_new_topic_valid_post_data(self):
        url = reverse('new_topic', kwargs={'pk': 1})
        data = {
            'subject': 'Test title',
            'message': 'Lorem ipsum dolor sit amet'
        }
        response = self.client.post(url, data)
        self.assertTrue(Topic.objects.exists())
        self.assertTrue(Post.objects.exists())

    def test_new_topic_invalid_post_data(self):
        '''
        Invalid post data should not redirect
        The expected behavior is to show the form again with validation errors
        '''
        url = reverse('new_topic', kwargs={'pk': 1})
        response = self.client.post(url, {})
        self.assertEquals(response.status_code, 200)

    def test_new_topic_invalid_post_data_empty_fields(self):
        '''
        Invalid post data should not redirect
        The expected behavior is to show the form again with validation errors
        '''
        url = reverse('new_topic', kwargs={'pk': 1})
        data = {
            'subject': '',
            'message': ''
        }
        response = self.client.post(url, data)
        self.assertEquals(response.status_code, 200)
        self.assertFalse(Topic.objects.exists())
        self.assertFalse(Post.objects.exists())

Ta có thể test tiếp cho việc gửi data 

`setUp`: tạo ra các objects giả định khi test (test xong sẽ biến mất)

`test_csrf`: vì CSRF TOKEN là phần quan trọng trong các requests POST, chúng ta phải tạo HTML chứa các token này

`test_new_topic_valid_post_data`: gửi các data thích hợp để check xem Topic và Post model có thể khởi tạo được không

`test_new_topic_invalid_post_data`: không gửi data check app

`test_new_topic_invalid_post_data_empty_fields`: gửi data rỗng check app

# Creating Forms The Right Way

- The Forms API is available in the module django.forms

Two types of forms: forms.Form and forms.ModelForm

- Form class is a general purpose form implementation

- A ModelForm is a subclass of Form, and it’s associated with a model class.

In [ ]:
from django import forms
from .models import Topic

class NewTopicForm(forms.ModelForm):
    # vị trí để thêm các field tùy chọn hoặc tùy chọn field
    # widget = ... để tùy chọn dạng field khi trình bày cho visitor
    # max_length = giới hạn kích thước`
    # help_text: đoạn text để hướng dẫn visitors
    message = forms.CharField(
        widget=forms.Textarea(
            attrs={'rows': 5, 'placeholder': 'What is on your mind?'}
        ),
        max_length=4000,
        help_text='The max length of the text is 4000.'
    )
    class Meta:
        model = Topic
        fields = ['subject', 'message']

In [ ]:
# views.py
from django.contrib.auth.models import User
from django.shortcuts import render, redirect, get_object_or_404
from .forms import NewTopicForm
from .models import Board, Topic, Post

def new_topic(request, pk):
    board = get_object_or_404(Board, pk=pk)
    user = User.objects.first()  # TODO: get the currently logged in user
    if request.method == 'POST':
        form = NewTopicForm(request.POST)
        if form.is_valid():
            topic = form.save(commit=False)
            topic.board = board
            topic.starter = user
            topic.save()
            post = Post.objects.create(
                message=form.cleaned_data.get('message'),
                topic=topic,
                created_by=user
            )
            return redirect('board_topics', pk=board.pk)  # TODO: redirect to the created topic page
    else:
        form = NewTopicForm()
    return render(request, 'new_topic.html', {'board': board, 'form': form})

Code html:

In [ ]:
{% extends 'base.html' %}

{% block title %}Start a New Topic{% endblock %}

{% block breadcrumb %}
  <li class="breadcrumb-item"><a href="{% url 'home' %}">Boards</a></li>
  <li class="breadcrumb-item"><a href="{% url 'board_topics' board.pk %}">{{ board.name }}</a></li>
  <li class="breadcrumb-item active">New topic</li>
{% endblock %}

{% block content %}
  <form method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit" class="btn btn-success">Post</button>
  </form>
{% endblock %}

`form` có 3 lựa chọn:
- form.as_table: sử dụng bảng các tags cho việc format input
- form.as_p: 
- form.as_ul:sử dụng HTML lists của các inputs 

If you see something like this:  Please fill out this field. when you submit the form, It's your browser doing a pre-validation. To disable it add the novalidate attribute to your form tag: `<form method="post" novalidate>`

Another important thing to note is that: there is no such a thing as "client-side validation." JavaScript validation or browser validation is just for usability purpose. And also to reduce the number of requests to the server. Data validation should always be done on the server side, where we have full control over the data.

# Rendering Bootstrap Forms

Install __django-widget-tweaks__



In [ ]:
pip install django-widget-tweaks

In [ ]:
# myproject/settings.py
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',

    'widget_tweaks',

    'boards',
]

In [ ]:
{% extends 'base.html' %}

{% load widget_tweaks %}

{% block title %}Start a New Topic{% endblock %}

{% block breadcrumb %}
  <li class="breadcrumb-item"><a href="{% url 'home' %}">Boards</a></li>
  <li class="breadcrumb-item"><a href="{% url 'board_topics' board.pk %}">{{ board.name }}</a></li>
  <li class="breadcrumb-item active">New topic</li>
{% endblock %}

{% block content %}
  <form method="post" novalidate>
    {% csrf_token %}

    {% for field in form %}
      <div class="form-group">
        {{ field.label_tag }}

        {% render_field field class="form-control" %}

        {% if field.help_text %}
          <small class="form-text text-muted">
            {{ field.help_text }}
          </small>
        {% endif %}
      </div>
    {% endfor %}

    <button type="submit" class="btn btn-success">Post</button>
  </form>
{% endblock %}

- First, we load it in the template by using the {% load widget_tweaks %}

` {% render_field field class="form-control" %} `

render_field thuộc app `django-widget-tweaks`

Để sử dụng nó ta hãy pass form `field` như một tham số, và sau đó add `HTML attributes` vào để complement nó

Một vài examples khác

In [ ]:
{% render_field form.subject class="form-control" %}
{% render_field form.message class="form-control" placeholder=form.message.label %}
{% render_field field class="form-control" placeholder="Write a message!" %}
{% render_field field style="font-size: 20px" %}

Ta sử dụng `boostrap 4 validation tags` bằng cách thay đổi thêm tí

In [ ]:
#templates/new_topic.html
<form method="post" novalidate>
  {% csrf_token %}

  {% for field in form %}
    <div class="form-group">
      {{ field.label_tag }}

      {% if form.is_bound %}
        {% if field.errors %}

          {% render_field field class="form-control is-invalid" %}
          {% for error in field.errors %}
            <div class="invalid-feedback">
              {{ error }}
            </div>
          {% endfor %}

        {% else %}
          {% render_field field class="form-control is-valid" %}
        {% endif %}
      {% else %}
        {% render_field field class="form-control" %}
      {% endif %}

      {% if field.help_text %}
        <small class="form-text text-muted">
          {{ field.help_text }}
        </small>
      {% endif %}
    </div>
  {% endfor %}

  <button type="submit" class="btn btn-success">Post</button>
</form>

## Reusable form templates

Ngoài ra, bạn có thể thực hiện việc kế thừa form template tạo sẵn

Tạo thư mục `includes` nằm trong folder templates

Thêm `form.html`

In [ ]:
{% load widget_tweaks %}

{% for field in form %}
  <div class="form-group">
    {{ field.label_tag }}

    {% if form.is_bound %}
      {% if field.errors %}
        {% render_field field class="form-control is-invalid" %}
        {% for error in field.errors %}
          <div class="invalid-feedback">
            {{ error }}
          </div>
        {% endfor %}
      {% else %}
        {% render_field field class="form-control is-valid" %}
      {% endif %}
    {% else %}
      {% render_field field class="form-control" %}
    {% endif %}

    {% if field.help_text %}
      <small class="form-text text-muted">
        {{ field.help_text }}
      </small>
    {% endif %}
  </div>
{% endfor %}

In [ ]:
#templates/new_topic.html
{% extends 'base.html' %}

{% block title %}Start a New Topic{% endblock %}

{% block breadcrumb %}
  <li class="breadcrumb-item"><a href="{% url 'home' %}">Boards</a></li>
  <li class="breadcrumb-item"><a href="{% url 'board_topics' board.pk %}">{{ board.name }}</a></li>
  <li class="breadcrumb-item active">New topic</li>
{% endblock %}

{% block content %}
  <form method="post" novalidate>
    {% csrf_token %}
    {% include 'includes/form.html' %}
    <button type="submit" class="btn btn-success">Post</button>
  </form>
{% endblock %}

Tag `{% include %}` được sử dụng để include HTML templates tạo sẵn cho form

The next form we implement, we can simply use `{% include 'includes/form.html' %}` to render it.

Cuối cùng, ta có thể tạo test cho các form

In [ ]:
from .forms import NewTopicForm

class NewTopicTests(TestCase):
    # ... other tests

    def test_contains_form(self):  # <- new test
        url = reverse('new_topic', kwargs={'pk': 1})
        response = self.client.get(url)
        form = response.context.get('form')
        self.assertIsInstance(form, NewTopicForm)

    def test_new_topic_invalid_post_data(self):  # <- updated this one
        '''
        Invalid post data should not redirect
        The expected behavior is to show the form again with validation errors
        '''
        url = reverse('new_topic', kwargs={'pk': 1})
        response = self.client.post(url, {})
        form = response.context.get('form')
        self.assertEquals(response.status_code, 200)
        self.assertTrue(form.errors)

Ta có thể sử dụng `assertIsIntansce` method check form được tạo

`self.assertTrue(form.errors)`: để đảm bảo form show ra errors khi data bị invalid